In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
# Generated by the protocol buffer compiler.  DO NOT EDIT!
# source: knn.proto

import sys
_b=sys.version_info[0]<3 and (lambda x:x) or (lambda x:x.encode('latin1'))
from google.protobuf import descriptor as _descriptor
from google.protobuf import message as _message
from google.protobuf import reflection as _reflection
from google.protobuf import symbol_database as _symbol_database
# @@protoc_insertion_point(imports)

_sym_db = _symbol_database.Default()




DESCRIPTOR = _descriptor.FileDescriptor(
  name='knn.proto',
  package='similarity',
  syntax='proto3',
  serialized_options=_b('\n\032com.google.step.similarityB\023OrganizationsProtos'),
  serialized_pb=_b('\n\tknn.proto\x12\nsimilarity\"\xbd\x01\n\rOrganizations\x12\x34\n\x04orgs\x18\x01 \x03(\x0b\x32&.similarity.Organizations.Organization\x1av\n\x0cOrganization\x12\n\n\x02id\x18\x01 \x01(\x05\x12\x42\n\tneighbors\x18\x02 \x03(\x0b\x32/.similarity.Organizations.Organization.Neighbor\x1a\x16\n\x08Neighbor\x12\n\n\x02id\x18\x01 \x01(\x05\x42\x31\n\x1a\x63om.google.step.similarityB\x13OrganizationsProtosb\x06proto3')
)




_ORGANIZATIONS_ORGANIZATION_NEIGHBOR = _descriptor.Descriptor(
  name='Neighbor',
  full_name='similarity.Organizations.Organization.Neighbor',
  filename=None,
  file=DESCRIPTOR,
  containing_type=None,
  fields=[
    _descriptor.FieldDescriptor(
      name='id', full_name='similarity.Organizations.Organization.Neighbor.id', index=0,
      number=1, type=5, cpp_type=1, label=1,
      has_default_value=False, default_value=0,
      message_type=None, enum_type=None, containing_type=None,
      is_extension=False, extension_scope=None,
      serialized_options=None, file=DESCRIPTOR),
  ],
  extensions=[
  ],
  nested_types=[],
  enum_types=[
  ],
  serialized_options=None,
  is_extendable=False,
  syntax='proto3',
  extension_ranges=[],
  oneofs=[
  ],
  serialized_start=193,
  serialized_end=215,
)

_ORGANIZATIONS_ORGANIZATION = _descriptor.Descriptor(
  name='Organization',
  full_name='similarity.Organizations.Organization',
  filename=None,
  file=DESCRIPTOR,
  containing_type=None,
  fields=[
    _descriptor.FieldDescriptor(
      name='id', full_name='similarity.Organizations.Organization.id', index=0,
      number=1, type=5, cpp_type=1, label=1,
      has_default_value=False, default_value=0,
      message_type=None, enum_type=None, containing_type=None,
      is_extension=False, extension_scope=None,
      serialized_options=None, file=DESCRIPTOR),
    _descriptor.FieldDescriptor(
      name='neighbors', full_name='similarity.Organizations.Organization.neighbors', index=1,
      number=2, type=11, cpp_type=10, label=3,
      has_default_value=False, default_value=[],
      message_type=None, enum_type=None, containing_type=None,
      is_extension=False, extension_scope=None,
      serialized_options=None, file=DESCRIPTOR),
  ],
  extensions=[
  ],
  nested_types=[_ORGANIZATIONS_ORGANIZATION_NEIGHBOR, ],
  enum_types=[
  ],
  serialized_options=None,
  is_extendable=False,
  syntax='proto3',
  extension_ranges=[],
  oneofs=[
  ],
  serialized_start=97,
  serialized_end=215,
)

_ORGANIZATIONS = _descriptor.Descriptor(
  name='Organizations',
  full_name='similarity.Organizations',
  filename=None,
  file=DESCRIPTOR,
  containing_type=None,
  fields=[
    _descriptor.FieldDescriptor(
      name='orgs', full_name='similarity.Organizations.orgs', index=0,
      number=1, type=11, cpp_type=10, label=3,
      has_default_value=False, default_value=[],
      message_type=None, enum_type=None, containing_type=None,
      is_extension=False, extension_scope=None,
      serialized_options=None, file=DESCRIPTOR),
  ],
  extensions=[
  ],
  nested_types=[_ORGANIZATIONS_ORGANIZATION, ],
  enum_types=[
  ],
  serialized_options=None,
  is_extendable=False,
  syntax='proto3',
  extension_ranges=[],
  oneofs=[
  ],
  serialized_start=26,
  serialized_end=215,
)

_ORGANIZATIONS_ORGANIZATION_NEIGHBOR.containing_type = _ORGANIZATIONS_ORGANIZATION
_ORGANIZATIONS_ORGANIZATION.fields_by_name['neighbors'].message_type = _ORGANIZATIONS_ORGANIZATION_NEIGHBOR
_ORGANIZATIONS_ORGANIZATION.containing_type = _ORGANIZATIONS
_ORGANIZATIONS.fields_by_name['orgs'].message_type = _ORGANIZATIONS_ORGANIZATION
DESCRIPTOR.message_types_by_name['Organizations'] = _ORGANIZATIONS
_sym_db.RegisterFileDescriptor(DESCRIPTOR)

Organizations = _reflection.GeneratedProtocolMessageType('Organizations', (_message.Message,), dict(

  Organization = _reflection.GeneratedProtocolMessageType('Organization', (_message.Message,), dict(

    Neighbor = _reflection.GeneratedProtocolMessageType('Neighbor', (_message.Message,), dict(
      DESCRIPTOR = _ORGANIZATIONS_ORGANIZATION_NEIGHBOR,
      __module__ = 'knn_pb2'
      # @@protoc_insertion_point(class_scope:similarity.Organizations.Organization.Neighbor)
      ))
    ,
    DESCRIPTOR = _ORGANIZATIONS_ORGANIZATION,
    __module__ = 'knn_pb2'
    # @@protoc_insertion_point(class_scope:similarity.Organizations.Organization)
    ))
  ,
  DESCRIPTOR = _ORGANIZATIONS,
  __module__ = 'knn_pb2'
  # @@protoc_insertion_point(class_scope:similarity.Organizations)
  ))
_sym_db.RegisterMessage(Organizations)
_sym_db.RegisterMessage(Organizations.Organization)
_sym_db.RegisterMessage(Organizations.Organization.Neighbor)


DESCRIPTOR._options = None
# @@protoc_insertion_point(module_scope)


In [ ]:
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub


module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

def embed(input):
  return model(input)

# Reduce logging output.
logging.set_verbosity(logging.ERROR)


In [17]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
import re
from sklearn.neighbors import NearestNeighbors
import string

# Reads csv as input and returns a Pandas DataFrame
def read_data(filename):
  df = pd.read_json(filename)
  # appends organization name to about information
  df.about += ' ' + df.name
  return df

# remove stopwords and punctuation, convert to lowercase
def process_text(input):
  # to lower case
  sentences = [word.lower() for word in input['about']]
  # remove punctuation
  punct_removed = []
  for sentence in sentences:
    punct_removed.append(''.join([s for s in sentence if s not in string.punctuation]))
  # Remove numbers
  num_removed = []
  for sentence in punct_removed:
    num_removed.append(re.sub(r'\d+', '', sentence))
  # remove stopwords and lemmatize
  nltk.download('stopwords')
  nltk.download('wordnet')
  stop_words = set(stopwords.words('english'))
  lemmatizer = WordNetLemmatizer()
  output = []
  for sentence in num_removed:
    words = []
    for word in sentence.split():
      if word not in stop_words:
        words.append(lemmatizer.lemmatize(word))
    output.append(' '.join(words))
  return output

def knn(features):
  # TODO: check the radius
  classifier = NearestNeighbors(n_neighbors=5, radius=0.4)
  classifier.fit(features)
  return classifier


def main():
  path = '/content/drive/My Drive/Capstone/'
  filename = path + 'processed_data/database.json'
  df = read_data(filename)
  text = process_text(df)  
  embeddings = embed(text)

  # K-Nearest Neighbors (k = 4)
  model = knn(embeddings)
  prediction = model.kneighbors(embeddings, 5, return_distance=False)
  orgs = Organizations()
  for i in range(df.shape[0]):
    print(df['name'][i], ': ', df['name'][prediction[i][1]], ', ',
        df['name'][prediction[i][2]], ', ', df['name'][prediction[i][3]], 
        ', ', df['name'][prediction[i][4]])
    org = orgs.orgs.add()
    org.id = df['id'][i]
    neighbor1 = org.neighbors.add()
    neighbor1.id = df['id'][prediction[i][1]]
    neighbor2 = org.neighbors.add()
    neighbor2.id = df['id'][prediction[i][2]]
    neighbor3 = org.neighbors.add()
    neighbor3.id = df['id'][prediction[i][3]]
    neighbor4 = org.neighbors.add()
    neighbor4.id = df['id'][prediction[i][4]]
  with open(path + 'knn/0_10000_neighbors.txt', 'wb') as out_file:
    out_file.write(orgs.SerializeToString())

In [ ]:
main()